In [10]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import joblib

# ✅ Load Dataset
df = pd.read_csv("merged_file.csv", encoding="latin1")

# ✅ Convert timestamp columns to datetime format if they exist
for col in df.columns:
    if "date" in col.lower() or "time" in col.lower():
        df[col] = pd.to_datetime(df[col], errors='coerce', dayfirst=True)

# ✅ Define required features
FEATURES = [
    "Z-Axis RMS Velocity (in/sec)", 
    "X-Axis RMS Velocity (in/sec)",
    "Temperature (°F)",
    "Z-Axis Peak Acceleration (G)",
    "X-Axis Peak Acceleration (G)",
    "Z-Axis Peak Velocity (mm/sec)",
    "X-Axis Peak Velocity (mm/sec)"
]

# ✅ Keep only available features
FEATURES = [col for col in FEATURES if col in df.columns]
if not FEATURES:
    raise ValueError("No valid features found in dataset!")

# ✅ Data Cleaning & Processing
def clean_numeric(value):
    """Cleans and converts sensor values to numeric format."""
    try:
        return float(str(value).strip().replace("..", ".").replace(",", ""))
    except ValueError:
        return np.nan
# Handle NaNs before scaling
df[FEATURES] = df[FEATURES].fillna(df[FEATURES].median())  # Recommended approach

# ✅ Feature Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[FEATURES])

# ✅ Train Isolation Forest model
model = IsolationForest(n_estimators=100, contamination=0.01, random_state=42)
model.fit(X_scaled)

# ✅ Save Model & Scaler
joblib.dump(model, "isolation_forest_model.pkl")
joblib.dump(scaler, "scaler1.pkl")

print("✅ Model & Scaler Saved Successfully!")


✅ Model & Scaler Saved Successfully!


In [4]:
#this is prototype model of vibration monitoring
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from datetime import datetime
import joblib
import time
from pymodbus.client import ModbusSerialClient

# Load the trained Isolation Forest model
model = joblib.load("isolation_forest_model.pkl")

# Modbus RTU Configuration
MODBUS_PORT = "COM7"  # Change based on your system
BAUDRATE = 19200
STOPBITS = 1
BYTESIZE = 8
PARITY = "N"
TIMEOUT = 3

# Initialize Modbus RTU Client
client = ModbusSerialClient(
    port=MODBUS_PORT,
    baudrate=BAUDRATE,
    stopbits=STOPBITS,
    bytesize=BYTESIZE,
    parity=PARITY,
    timeout=TIMEOUT
)

# Register Addresses (Subtract 1 for pymodbus)
REGISTER_MAPPING = {
    "Z-Axis RMS Velocity (in/sec)": 45201 - 40001,
    "X-Axis RMS Velocity (in/sec)": 45205 - 40001,
    "Temperature (°F)": 45203 - 40001,
    "Z-Axis Peak Acceleration (G)": 45207 - 40001,
    "X-Axis Peak Acceleration (G)": 45208 - 40001,
    "Z-Axis Peak Velocity (mm/sec)": 45218 - 40001,
    "X-Axis Peak Velocity (mm/sec)": 45220 - 40001,
}

# Scaling Factors
SCALING_FACTORS = {
    "Z-Axis RMS Velocity (in/sec)": 6.5535 / 65535,
    "X-Axis RMS Velocity (in/sec)": 6.5535 / 65535,
    "Temperature (°F)": 327.67 / 32767,
    "Z-Axis Peak Acceleration (G)": 65.535 / 65535,
    "X-Axis Peak Acceleration (G)": 65.535 / 65535,
    "Z-Axis Peak Velocity (mm/sec)": 65.535 / 65535,
    "X-Axis Peak Velocity (mm/sec)": 65.535 / 65535,
}

def read_modbus_data():
    """ Reads sensor data from Modbus RTU """
    if not client.connect():
        print("❌ Failed to connect to Modbus RTU device")
        return None
    
    live_data = {}
    for feature, address in REGISTER_MAPPING.items():
        try:
            response = client.read_holding_registers(address=address, count=1, slave=1)
            if response and response.registers:
                raw_value = response.registers[0]
                live_data[feature] = round(raw_value * SCALING_FACTORS[feature], 3)
            else:
                live_data[feature] = None
        except Exception as e:
            print(f"Error reading {feature}: {e}")
            live_data[feature] = None
    return live_data

def detect_anomaly(live_data):
    if not live_data:
        return "Error"
    live_df = pd.DataFrame([live_data], columns=REGISTER_MAPPING.keys())
    prediction = model.predict(live_df)
    return "Anomaly" if prediction[0] == -1 else "Normal"

def monitor_live_data():
    print("✅ Monitoring live data from Modbus RTU...")
    while True:
        live_data = read_modbus_data()
        if not live_data or None in live_data.values():
            print("⚠️ Error reading Modbus data, skipping this cycle.")
            time.sleep(2)
            continue
        
        result = detect_anomaly(live_data)
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        if result == "Anomaly":
            print(f"[{timestamp}] 🚨 ALERT! Anomaly detected: {live_data}")
        else:
            print(f"[{timestamp}] ✅ Normal data: {live_data}")
        
        time.sleep(2)  # Monitoring interval

# Start live monitoring
monitor_live_data()

client.close()  # Ensure Modbus client is closed before re-running


✅ Monitoring live data from Modbus RTU...
[2025-02-11 10:50:41] ✅ Normal data: {'Z-Axis RMS Velocity (in/sec)': 0.021, 'X-Axis RMS Velocity (in/sec)': 0.034, 'Temperature (°F)': 89.94, 'Z-Axis Peak Acceleration (G)': 0.038, 'X-Axis Peak Acceleration (G)': 0.038, 'Z-Axis Peak Velocity (mm/sec)': 0.741, 'X-Axis Peak Velocity (mm/sec)': 1.214}
[2025-02-11 10:50:43] ✅ Normal data: {'Z-Axis RMS Velocity (in/sec)': 0.021, 'X-Axis RMS Velocity (in/sec)': 0.034, 'Temperature (°F)': 89.94, 'Z-Axis Peak Acceleration (G)': 0.038, 'X-Axis Peak Acceleration (G)': 0.038, 'Z-Axis Peak Velocity (mm/sec)': 0.741, 'X-Axis Peak Velocity (mm/sec)': 1.214}
[2025-02-11 10:50:45] ✅ Normal data: {'Z-Axis RMS Velocity (in/sec)': 0.022, 'X-Axis RMS Velocity (in/sec)': 0.031, 'Temperature (°F)': 89.94, 'Z-Axis Peak Acceleration (G)': 0.038, 'X-Axis Peak Acceleration (G)': 0.042, 'Z-Axis Peak Velocity (mm/sec)': 0.8, 'X-Axis Peak Velocity (mm/sec)': 1.114}
[2025-02-11 10:50:47] ✅ Normal data: {'Z-Axis RMS Velocit

KeyboardInterrupt: 